<a href="https://colab.research.google.com/github/Kamal-018/Sentiment_Analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
# configuring the path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# API to fecth dataset from the kaggle
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 83% 67.0M/80.9M [00:00<00:00, 180MB/s]
100% 80.9M/80.9M [00:00<00:00, 177MB/s]


In [ ]:
# extracting from the compressed zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('dataset extracted')

dataset extracted


In [ ]:
# importing the required libraries
import numpy as np
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#reading the csv file
tweets = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
tweets.shape

(1599999, 6)

In [ ]:
tweets.head(1)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [ ]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
tweets = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names =column_names, encoding='ISO-8859-1')

In [ ]:
tweets.head(1)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."


In [ ]:
#dropping the null datasets
tweets.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
tweets['target'].value_counts()  #balance dataset

,count
target,
0,800000
4,800000


In [ ]:
tweets.replace({'target':{4:1}}, inplace=True)

In [ ]:
tweets['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
#stemming process to find out the root words
english_stopwords = set(stopwords.words('english'))

def stem_text(text):

    # remove non-alphabetic characters and convert to lowercase
    stemmed_text = re.sub('[^A-Za-z]', ' ', text)
    stemmed_text = stemmed_text.lower()

    # tokenize the tweet
    stemmed_text = stemmed_text.split()

    # stem each token without stemming the stopwords
    stemmed_tokens = []
    for token in stemmed_text:
        if token not in english_stopwords:
            stemmed_token = PorterStemmer().stem(token)
            stemmed_tokens.append(stemmed_token)

    stemmed_text = stemmed_tokens

    # Join the stemmed words back into a single string
    stemmed_text = ' '.join(stemmed_text)

    return stemmed_text


In [ ]:
tweets['text'] = tweets['text'].apply(stem_text)

KeyboardInterrupt: 